# Homework 3.1 - Python packages and virtual environments 101


## Commonly used libraries in the Standard Library

The Python Standard Library is quite extensive and consists of more than 200 core modules. These modules cover a wide range of functionalities, from file I/O and system calls, to internet protocols - some more useful, or more commonly used, than others.

Below we'll see a basic, example of some of the libraries that are commonly used.

In [ ]:
# Import the required modules from the Python Standard Library
import datetime
import os
import platform
import random
import math
import json

# Get the current date and time
now = datetime.datetime.now()

# Get the current working directory
cwd = os.getcwd()

# Get the current platform
current_platform = platform.system()

# Generate a random number
random_number = random.randint(1, 10)

# Calculate the square root of the random number
sqrt = math.sqrt(random_number)

# Create a dictionary with the results
results = {
    "date_time": str(now),
    "directory": cwd,
    "platform": current_platform,
    "random_number": random_number,
    "square_root": sqrt
}

# Convert the dictionary to a JSON string
json_results = json.dumps(results, indent=4)

# Print the JSON string
print("Hello, here are some fun facts for you in JSON format:")
print(json_results)

## Virtual environments

Now Azure ML has a few conda environments already created for us... head over to the terminal of your running compute instance and run `conda env list` to see the full list. As we covered previously, each of these virtual environments will have a collection of different packages and versions of Python (some even slated as end-of-life...)


To change your current environment, you can run the command `conda activate <insert_env_name_here>` which will 'deactivate' the current environment and then 'activate' the new environment. Once this has completed, go ahead and run the following command `conda list` - this will print all of the packages installed within the environment you have just activated.

Now to create a new conda environment... The command `conda create -n myenv python=3.9 scipy=0.17.3 astroid babel` will create a new environment with the name 'myenv', with Python v3.9 and a few packages installed. Note that after creating the environment, you will need to run the 'activate' command in order to use it.

Make sense?! With the fundaments covered, you can now go ahead and create a new virtual environment with Python v3.11...

Once that has been completed, we have a few more steps to follow...

When we run our notebooks, we use one of the kernels found in the top right of the notebook - but how do can we use our newly created virtual environment in a notebook?

A kernel, while very similar to a virtual environment, and can even be associated with a specific virtual environment, is used as a computational engine specifically to run cells within a notebook. To convert our virtual environment into a kernel, we can run the following commands:

`pip install ipykernel`

`python -m ipykernel install --user --name <env_name_here> --display-name "Python openai_env"`

## 